### Day 6

This one has a very interesting math problem involving exponential growth. We're given these details about some fish that are reproducing and, apparently, never dying

1. An adult lantern fish reproduces every 7 days
2. A juvenile lantern fish requires 2 days to mature, and then reproduces every 7 days
3. We begin with a number of lantern fish at different points into their reproductive cycle

Some observations:
- We can reduce the calculations by just condsidering all the different cases and then multiplying the results by the frequencies
- It might be best to center them around their first reproduction and then just offset by the starting value 
- It isn't as simple as writing an exponential growth equation since there is an additional lag that each generation has...

In [15]:
import pandas as pd
import numpy as np

with open('d6.txt') as file:
    puzzle_input = file.readline()

fish_counts = puzzle_input.replace('\n', '')
fish_counts = fish_counts.split(',')
fish_counts = pd.Series(fish_counts).astype('int')
fish_counts = (
    fish_counts
    .value_counts()
    .reset_index(drop = False)
    .rename(columns = {'index' : 'timer', 0 : 'N'})
    .sort_values('timer')
)
fish_counts['start_date'] = 80-fish_counts['timer']
fish_counts

,timer,N,start_date
0,1,116,79
3,2,45,78
4,3,42,77
2,4,48,76
1,5,49,75


Alright, let's set that aside for a while and just model the fish at `timer = 0` and record the counts at each day.

In [16]:
# List of fish timers:
fish_list = [0]
# Counts of fish
fish_n = [1]
# This will store how the fish change after one day:
growth_dict = dict(
    zip(
        list(range(9)),
        [6, 0, 1, 2, 3, 4, 5, 6, 7]
    )
)

for i in range(80):
    
    # New fish:
    new_fish = [8 for x in fish_list if x == 0]
    
    # Change fish:
    fish_list = [growth_dict[x] for x in fish_list]
    fish_list += new_fish

    fish_n.append(len(fish_list))

fish_df = pd.DataFrame({
    'day' : list(range(len(fish_n))),
    'fish_n' : fish_n
})

fish_df.head(11)

,day,fish_n
0,0,1
1,1,2
2,2,2
3,3,2
4,4,2
5,5,2
6,6,2
7,7,2
8,8,3
9,9,3


Cool, now we just need to join this to our counts and take a weighted sum!

In [17]:
fish_counts = pd.merge(
    left = fish_counts,
    right = fish_df,
    how = 'left',
    left_on = 'start_date',
    right_on = 'day'
)

(fish_counts['fish_n']*fish_counts['N']).sum()

360761

### Part 2:

Same thing but with 256 days instead of 8. Thought I could just re-run this but not so! Exponential growth is strong. Instead, let's condense this to a data frame to store the counts. This is way smarter anyways.

In [19]:
# drop these columns 
fish_counts.drop(columns = ['start_date', 'day', 'fish_n'], inplace = True)
fish_counts['start_date'] = 256-fish_counts['timer']

# Initial state
fish_list = pd.Series([1, 0, 0, 0, 0, 0, 0, 0, 0])
# Counts of fish
fish_n = [1]

for i in range(256):
    
    # New fish:
    new_fish = fish_list.iloc[0]
    
    # Change fish:
    fish_list = fish_list.shift(-1)
    
    # Add new fish and reset adults from 0 to 6
    fish_list.iloc[8] = new_fish
    fish_list.iloc[6] += new_fish

    fish_n.append(fish_list.sum())

fish_df = pd.DataFrame({
    'day' : list(range(len(fish_n))),
    'fish_n' : fish_n
})

fish_counts = pd.merge(
    left = fish_counts,
    right = fish_df,
    how = 'left',
    left_on = 'start_date',
    right_on = 'day'
)

(fish_counts['fish_n']*fish_counts['N']).sum()

1632779838045.0

Guess I'm not surprised that the more mathematical ones are easier for me